In [2]:
!pip install earthengine-api



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached earthengine_api-1.5.3-py3-none-any.whl.metadata (2.1 kB)
  Using cached google_cloud_storage-3.0.0-py2.py3-none-any.whl.metadata (12 kB)
  Using cached google_api_python_client-2.161.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_api_core-2.24.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_cloud_core-2.4.2-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.6.0-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached googleapis_c

In [2]:
# Import packages
import ee

# Authenticate and initialize GEE
ee.Authenticate()  # Follow the prompts to log in and authorize
ee.Initialize()



Successfully saved authorization token.


In [ ]:

# Define ROI for Punjab, India
roi = ee.Geometry.Rectangle([74.5, 29.5, 76.5, 31.5])  # [min_lon, min_lat, max_lon, max_lat]

# Define time range (e.g., for Rabi season 2023)
start_date = '2023-01-01'
end_date = '2023-03-31'

# Load Sentinel-2 collection
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Filter by cloud cover

# Apply cloud masking
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # Bitwise operation to mask clouds
    return image.updateMask(cloud_mask)

collection = collection.map(mask_clouds)

# Export NDVI image (median composite)
task = ee.batch.Export.image.toDrive(
    image=collection.median(),  # Median composite of the collection
    description='Sentinel2_Punjab_2023',
    scale=10,  # 10m resolution
    region=roi,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

# Start the export task
task.start()

# Check task status
print(task.status())

c:\Users\Priyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


{'state': 'READY', 'description': 'Sentinel2_Punjab_2023', 'priority': 100, 'creation_timestamp_ms': 1740221206147, 'update_timestamp_ms': 1740221206147, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SZHRZ3QXLMZBORKE7W4LFUSY', 'name': 'projects/730570924758/operations/SZHRZ3QXLMZBORKE7W4LFUSY'}
